# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
#langchain library
!pip install langchain==0.0.350
#LLM
!pip install accelerate==0.25.0
!pip install transformers==4.36.2
!pip install bitsandbytes==0.41.2
#Text Embedding
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1
#vectorstore
!pip install pymupdf==1.23.8
!pip install faiss-gpu==1.7.2
!pip install faiss-cpu==1.7.4

  Using cached langchain-0.0.350-py3-none-any.whl.metadata (13 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.32-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl.metadata (8.4 kB)
  Using cached langchain_community-0.0.30-py3-none-any.whl.metadata (8.4 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_community-0.0.28-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_community-0.0.27-py3-none-any.whl.metadata (8.2 kB)
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.26-py3-none-any.whl.metadata (8.2 kB)
  Using c

In [3]:
import os
import torch
# Set GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [4]:
from langchain import PromptTemplate

prompt_template = """
Welcome to AITBot, your helpful guide to the Asian Institute of Technology!
Have questions about AIT? I've got the answers!
From programs and courses to campus life and facilities,
I'm here to provide you with all the information you need.
Don't hesitate to ask me anything!
{context}
Question: {question}
Answer:
""".strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="Welcome to AITBot, your helpful guide to the Asian Institute of Technology!\nHave questions about AIT? I've got the answers!\nFrom programs and courses to campus life and facilities,\nI'm here to provide you with all the information you need.\nDon't hesitate to ask me anything!\n{context}\nQuestion: {question}\nAnswer:")

In [5]:
PROMPT.format(
    context = "The Asian Institute of Technology (AIT) offers a wide range of master's and doctoral programs in engineering, technology, environment, and management.",
    question = "What kind of programs does AIT offer?"
)


"Welcome to AITBot, your helpful guide to the Asian Institute of Technology!\nHave questions about AIT? I've got the answers!\nFrom programs and courses to campus life and facilities,\nI'm here to provide you with all the information you need.\nDon't hesitate to ask me anything!\nThe Asian Institute of Technology (AIT) offers a wide range of master's and doctoral programs in engineering, technology, environment, and management.\nQuestion: What kind of programs does AIT offer?\nAnswer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [11]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = 'AIT-BROCHURE.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [12]:
documents

[Document(page_content='Apart from academic programs in three \nAIT Schools, these Outreach Centers are \nthe connectors for AIT’s education and re-\nsearch outreach with industry and society.\nAIT Extension\n--Executive education, lifelong learning, skill \ndevelopment through short-courses, trainings, \nworkshops, and seminars\nAIT Solutions\n- - Innovative consulting/solutions in \nengineering, technology, environment, \ninfrastructure, education and management\nGeoinformatics Center \n--Consulting and capacity building in \napplication of Remote Sensing and \nGeoinformatics in agriculture, disaster risk \nmanagement, and environment\nRegional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment \nand sustainable development goals \nthrough research and capacity building\nBelt & Road Research Center \n--Research and studies in sustainability \nissues of the Belt and Road region\nInternet Education and Research \nLaboratory \n--Development, training, and edu

In [13]:
len(documents)

2

In [14]:
documents[1]

Document(page_content='Department of Food, \nAgriculture and Bioresources\nAgribusiness Management \nAgricultural Systems and \nEngineering \nAquaculture and Aquatic \nResources Management \nFood Engineering and \nBioprocess Technology \nFood Innovation, Nutrition and \nHealth\nS T U DY  O P T I O N S  A T  A I T\nMaster’s and PhD Degree Programs\nAvailable in many fields of specializations across its \nthree Schools, covering various aspects of techno-\nlogy, social developments, and management. Sev-\neral new degree programs are designed to address \nemerging global challenges and to respond to cur-\nrent demands of the workforce. \nFlexible Degree Options\nNew alternative to traditional “On-campus Degree \nOptions”. Students can do many courses online via \nAIT’s Hybrid Modes of Instruction, with an option of \nflexible payment: course-by-course. The minimum \nenrollment is two courses per semester – ideal for those \nnot wanting to take too much time away from career, \nor those wa

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [16]:
doc[1]

Document(page_content='Regional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment \nand sustainable development goals \nthrough research and capacity building\nBelt & Road Research Center \n--Research and studies in sustainability \nissues of the Belt and Road region\nInternet Education and Research \nLaboratory \n--Development, training, and education \nprograms related to internet development \nand IT topics\nAIT Artificial Intelligence\nTechnology Center\n- - R e s e a r c h  a n d  d eve l o p m e n t  t o \nincorporate AI into AIT programs and its \napplication to real-world problems \nYunus Center at AIT\n- - E nte r p r i s e  s o l u t i o n s fo r  S D G s &', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G165) Quartz PDFContext', 'cre

In [17]:
len(doc)

25

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [18]:
import torch
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# Define the device (CPU or CUDA GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device}
)


/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [22]:
# %cd /content/drive/My Drive/A7

In [23]:
#locate vectorstore
# %cd /content/drive/My Drive/A7

vector_path = './vector-store'
import os
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('Created path:', vector_path)

In [24]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [25]:
#calling vector from local
vector_path = './vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [26]:
#ready to use
retriever = vectordb.as_retriever()

In [27]:
retriever.get_relevant_documents("What are the courses at AIT?")

[Document(page_content='AIT offers an opportunity to all students \nenrolling in Master’s degree programs \nto take internships in national and \nmultinational enterprises as a part of \nthe academic curriculum. The internship \ncarries 12 credits and duration can \nrange from 4 to 6 months. The internship \nwork generally relates to the academic \nprogram and the research area of the \nstudents, and includes student-reports \non the work done during the internship. \nThis provides the students practical \nexperiences, exposure, networking and \njob placement opportunities in future.\nMinor \nSeveral programs at AIT also offer minors. \nA minor will complement the knowledge \nacquired in the major program and can', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G165) Quartz

In [28]:
retriever.get_relevant_documents("Where is the Location of AIT?")

[Document(page_content='Apart from academic programs in three \nAIT Schools, these Outreach Centers are \nthe connectors for AIT’s education and re-\nsearch outreach with industry and society.\nAIT Extension\n--Executive education, lifelong learning, skill \ndevelopment through short-courses, trainings, \nworkshops, and seminars\nAIT Solutions\n- - Innovative consulting/solutions in \nengineering, technology, environment, \ninfrastructure, education and management\nGeoinformatics Center \n--Consulting and capacity building in \napplication of Remote Sensing and \nGeoinformatics in agriculture, disaster risk \nmanagement, and environment\nRegional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [29]:
pip install langchain==0.1.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.350
    Uninstalling langchain-0.0.350:
      Successfully uninstalled langchain-0.0.350
Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [31]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [32]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [33]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [34]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [35]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [36]:
%cd /content/drive/My Drive/A7

!git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

[Errno 2] No such file or directory: '/content/drive/My Drive/A7'
/home/dlb/Downloads/A7-20240426T175403Z-002/A7
fatal: destination path 'fastchat-t5-3b-v1.0' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [39]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [40]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [41]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [42]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [43]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Welcome to AITBot, your helpful guide to the Asian Institute of Technology!\nHave questions about AIT? I've got the answers!\nFrom programs and courses to campus life and facilities,\nI'm here to provide you with all the information you need.\nDon't hesitate to ask me anything!\n{context}\nQuestion: {question}\nAnswer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x73af2a6fd390>)), document_variable_name='context')

In [44]:
query = "Where is the Location of AIT?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Welcome to AITBot, your helpful guide to the Asian Institute of Technology!
Have questions about AIT? I've got the answers!
From programs and courses to campus life and facilities,
I'm here to provide you with all the information you need.
Don't hesitate to ask me anything!
Apart from academic programs in three 
AIT Schools, these Outreach Centers are 
the connectors for AIT’s education and re-
search outreach with industry and society.
AIT Extension
--Executive education, lifelong learning, skill 
development through short-courses, trainings, 
workshops, and seminars
AIT Solutions
- - Innovative consulting/solutions in 
engineering, technology, environment, 
infrastructure, education and management
Geoinformatics Center 
--Consulting and capacity building in 
application of Remote Sensing and 
Geoinformatics in agriculture, disaster risk 
management, and environment
Regional Resou

{'input_documents': [Document(page_content='Apart from academic programs in three \nAIT Schools, these Outreach Centers are \nthe connectors for AIT’s education and re-\nsearch outreach with industry and society.\nAIT Extension\n--Executive education, lifelong learning, skill \ndevelopment through short-courses, trainings, \nworkshops, and seminars\nAIT Solutions\n- - Innovative consulting/solutions in \nengineering, technology, environment, \ninfrastructure, education and management\nGeoinformatics Center \n--Consulting and capacity building in \napplication of Remote Sensing and \nGeoinformatics in agriculture, disaster risk \nmanagement, and environment\nRegional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Ver

In [45]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Welcome to AITBot, your helpful guide to the Asian Institute of Technology!\nHave questions about AIT? I've got the answers!\nFrom programs and courses to campus life and facilities,\nI'm here to provide you with all the information you need.\nDon't hesitate to ask me anything!\n{context}\nQuestion: {question}\nAnswer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x73af2a6fd390>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase th

## 5. Chatbot

In [50]:
prompt_question = "What are the STUDENT OPPORTUNITIES at AIT?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Where is the Location of AIT?'), AIMessage(content='<pad>  The  Asian  Institute  of  Technology  is  located  in  Bangkok,  Thailand.\n'), HumanMessage(content='What courses does AIT offer'), AIMessage(content='<pad> < pad>\n The  courses  offered  by  AIT  include:\n *  Engineering  courses\n *  Computer  Science  courses\n *  Electrical  Engineering  courses\n *  Mechanical  Engineering  courses\n *  Civil  Engineering  courses\n *  Environmental  Engineering  courses\n *  Environmental  Science  courses\n *  Environmental  Management  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'What are the STUDENT OPPORTUNITIES at AIT?',
 'chat_history': [HumanMessage(content='Where is the Location of AIT?'),
  AIMessage(content='<pad>  The  Asian  Institute  of  Technology  is  located  in  Bangkok,  Thailand.\n'),
  HumanMessage(content='What courses does AIT offer'),
  AIMessage(content='<pad> < pad>\n The  courses  offered  by  AIT  include:\n *  Engineering  courses\n *  Computer  Science  courses\n *  Electrical  Engineering  courses\n *  Mechanical  Engineering  courses\n *  Civil  Engineering  courses\n *  Environmental  Engineering  courses\n *  Environmental  Science  courses\n *  Environmental  Management  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Envi

In [51]:
prompt_question = "Where is the Location of AIT?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What courses does AIT offer'), AIMessage(content='<pad> < pad>\n The  courses  offered  by  AIT  include:\n *  Engineering  courses\n *  Computer  Science  courses\n *  Electrical  Engineering  courses\n *  Mechanical  Engineering  courses\n *  Civil  Engineering  courses\n *  Environmental  Engineering  courses\n *  Environmental  Science  courses\n *  Environmental  Management  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environme

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Welcome to AITBot, your helpful guide to the Asian Institute of Technology!
Have questions about AIT? I've got the answers!
From programs and courses to campus life and facilities,
I'm here to provide you with all the information you need.
Don't hesitate to ask me anything!
Apart from academic programs in three 
AIT Schools, these Outreach Centers are 
the connectors for AIT’s education and re-
search outreach with industry and society.
AIT Extension
--Executive education, lifelong learning, skill 
development through short-courses, trainings, 
workshops, and seminars
AIT Solutions
- - Innovative consulting/solutions in 
engineering, technology, environment, 
infrastructure, education and management
Geoinformatics Center 
--Consulting and capacity building in 
application of Remote Sensing and 
Geoinformatics in agriculture, disaster risk 
management, and environ

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'Where is the Location of AIT?',
 'chat_history': [HumanMessage(content='What courses does AIT offer'),
  AIMessage(content='<pad> < pad>\n The  courses  offered  by  AIT  include:\n *  Engineering  courses\n *  Computer  Science  courses\n *  Electrical  Engineering  courses\n *  Mechanical  Engineering  courses\n *  Civil  Engineering  courses\n *  Environmental  Engineering  courses\n *  Environmental  Science  courses\n *  Environmental  Management  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *  Environmental  Science  courses\n *

In [48]:
prompt_question = "What courses does AIT offer"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  AITBot  is  a  virtual  assistant  created  by  AIT  to  provide  information  and  answer  questions  about  the  Asian  Institute  of  Technology.  I  am  designed  to  assist  students,  parents,  and  alumni  with  their  questions  about  AIT.  I  am  a  language  model  that  can  understand  and  respond  to  a  wide  range  of  questions  about  AIT.  I  am  here  to  help  you  with  any  questions  you  may  have  about  the  school  and  its  programs.  If  you  have  any  specific  questions  about  AIT,  feel  free  to  ask  me  and  I  will  do  my  best  to  provide  you  with  the  information  you  need.\n'), HumanMessage(c

{'question': 'What courses does AIT offer',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  AITBot  is  a  virtual  assistant  created  by  AIT  to  provide  information  and  answer  questions  about  the  Asian  Institute  of  Technology.  I  am  designed  to  assist  students,  parents,  and  alumni  with  their  questions  about  AIT.  I  am  a  language  model  that  can  understand  and  respond  to  a  wide  range  of  questions  about  AIT.  I  am  here  to  help  you  with  any  questions  you  may  have  about  the  school  and  its  programs.  If  you  have  any  specific  questions  about  AIT,  feel  free  to  ask  me  and  I  will  do  my  best  to  provide  you  with  the  information  you  need.\n'),
  HumanMessage(content='Where is the Location of AIT?'),
  AIMessage(content='<pad>  The  Asian  Institute  of  Technology  is  located  in  Bangkok,  Thailand.\n')],
 'answer': '<pad> < pad>\n The  courses  offered  by  AIT  i

In [52]:
prompt_question = "What facilities and services does AIT provide to its students?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What facilities and services does AIT provide to its students?'), AIMessage(content='<pad> < pad>\n AIT  offers  the  following  facilities  and  services  to  its  students:\n 1.  Library:  AIT  has  a  large  library  with  over  30,000  books  and  journals,  as  well  as  access  to  online  resources.\n 2.  Computer  Labs:  AIT  has  several  computer  labs  equipped  with  the  latest  technology  and  software  for  research  and  academic  purposes.\n 3.  Sports  Facilities:  AIT  has  a  sports  center  with  a  swimming  pool,  gym,  and  tennis  courts.\n 4.  Library  Services:  AIT  has  a  library  service  that  provides  access  to  a  wide  range  of  books,  journals,  and  d

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Welcome to AITBot, your helpful guide to the Asian Institute of Technology!
Have questions about AIT? I've got the answers!
From programs and courses to campus life and facilities,
I'm here to provide you with all the information you need.
Don't hesitate to ask me anything!
Apart from academic programs in three 
AIT Schools, these Outreach Centers are 
the connectors for AIT’s education and re-
search outreach with industry and society.
AIT Extension
--Executive education, lifelong learning, skill 
development through short-courses, trainings, 
workshops, and seminars
AIT Solutions
- - Innovative consulting/solutions in 
engineering, technology, environment, 
infrastructure, education and management
Geoinformatics Center 
--Consulting and capacity building in 
application of Remote Sensing and 
Geoinformatics in agriculture, disaster risk 
management, and environ

/home/dlb/anaconda3/envs/NLU/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'What facilities and services does AIT provide to its students?',
 'chat_history': [HumanMessage(content='What facilities and services does AIT provide to its students?'),
  AIMessage(content='<pad> < pad>\n AIT  offers  the  following  facilities  and  services  to  its  students:\n 1.  Library:  AIT  has  a  large  library  with  over  30,000  books  and  journals,  as  well  as  access  to  online  resources.\n 2.  Computer  Labs:  AIT  has  several  computer  labs  equipped  with  the  latest  technology  and  software  for  research  and  academic  purposes.\n 3.  Sports  Facilities:  AIT  has  a  sports  center  with  a  swimming  pool,  gym,  and  tennis  courts.\n 4.  Library  Services:  AIT  has  a  library  service  that  provides  access  to  a  wide  range  of  books,  journals,  and  databases.\n 5.  Career  Services:  AIT  has  a  career  center  that  provides  information  and  guidance  to  students  on  career  planning,  job  search,  and  networking.\n 

## Task 2

## 1) Analyze the model’s performance in retrieving information. (2 points)


**Evaluation and Recommendations:**

**Handling Clear Queries:**
The bot’s ability to respond accurately to clear questions like "Who are you?" and specific queries about the location or courses offered by AIT suggests that the retrieval mechanisms and source documents are appropriately aligned for such inquiries.
Ensuring that the documents are comprehensive and well-organized will help maintain or improve performance for these types of queries.

**Handling Ambiguous Queries:**
For ambiguous or poorly formulated queries, the bot’s performance can be erratic. This is often due to the model's limitations in understanding nuanced human language or the absence of clear information in the source documents to match the query intent.
Implementing a clarification mechanism where the bot asks follow-up questions to refine the user’s query could significantly enhance accuracy. This approach helps narrow down the user's intent and provides the model with a clearer direction for retrieving or generating the right information.

## - Address any issues related to the model providing unrelated information. (2 points)

**Model Training and Fine-Tuning:**
Further refining the training data and continuously fine-tuning the model’s algorithms can improve its ability to interpret and respond to complex queries. This might involve training on a broader range of topics or incorporating more diverse language structures into the training dataset.
Regular updates and adjustments based on user feedback and interaction logs can also help identify and address specific areas where the bot underperforms.

**Performance Metrics and Monitoring:**
Establishing clear metrics for measuring the bot's performance (e.g., accuracy, relevance, user satisfaction) is crucial. These metrics can guide systematic improvements and help benchmark the bot’s capabilities over time.
Continuous monitoring through logs and user feedback can provide ongoing insights into how well the bot meets user expectations and areas where it may require adjustments.